In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install -q pytorch-lightning wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import pytorch_lightning as pl

In [ ]:
import torch
import torchvision.transforms as T
import torchvision.datasets
import numpy as np
from torch.utils.data import Subset


def get_train_data(conf):
    if conf.dataset.name == 'mnist':

        transform = T.Compose(
            [
                #T.RandomHorizontalFlip(),
                T.ToTensor(),
                #T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
            ]
        )
        transform_test = T.Compose(
            [
                T.ToTensor(),
                #T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
            ]
        )

        train_set = torchvision.datasets.MNIST(conf.dataset.path,
                                                 train=True,
                                                 transform=transform,
                                                 download=True)
        valid_set = torchvision.datasets.MNIST(conf.dataset.path,
                                                  train=True,
                                                  transform=transform_test,
                                                  download=True)

        num_train  = len(train_set)
        indices    = torch.randperm(num_train).tolist()
        valid_size = int(np.floor(0.05 * num_train))

        train_idx, valid_idx = indices[valid_size:], indices[:valid_size]

        train_set = Subset(train_set, train_idx)
        valid_set = Subset(valid_set, valid_idx)


        test_set = torchvision.datasets.MNIST(conf.dataset.path,
                                                 train=False,
                                                 transform=transform_test,
                                                 download=True)
   
    else:
        raise FileNotFoundError

    return train_set, valid_set, test_set

In [ ]:
import torch
import numpy as np
from scipy import integrate

class VariancePreservingSDE(torch.nn.Module):

    def __init__(self, beta_min=0.1, beta_max=20.0, T=1.0, t_epsilon=0.001):
        super().__init__()
        self.beta_min = beta_min
        self.beta_max = beta_max
        self.T = T
        self.t_epsilon = t_epsilon

    def beta(self, t):
        return self.beta_min + (self.beta_max-self.beta_min)*t

    def mean_weight(self, t):
        return torch.exp(-0.25 * t**2 * (self.beta_max-self.beta_min) - 0.5 * t * self.beta_min)

    def var(self, t):
        return 1. - torch.exp(-0.5 * t**2 * (self.beta_max-self.beta_min) - t * self.beta_min)
    
    def marginal_prob_std(self, t):
       return self.var(t) ** 0.5 
       
    def f(self, t, y):
        return - 0.5 * self.beta(t) * y

    def g(self, t, y):
        beta_t = self.beta(t)
        return beta_t**0.5

    def sample(self, t, y0, return_noise=False):
      
        mu = self.mean_weight(t) * y0
        std = self.var(t) ** 0.5
        epsilon = torch.randn_like(y0)
        yt = epsilon * std + mu
        if not return_noise:
            return yt
        else:
            return yt, epsilon, std, self.g(t, yt)


error_tolerance = 1e-5
def ode_sampler(score_model,
                batch_size=64, 
                atol=error_tolerance, 
                rtol=error_tolerance, 
                device='cuda', 
                z=None,
                eps=1e-3):
    
    T1 = torch.nn.Parameter(torch.FloatTensor([1.0]), requires_grad=False)
        
    sde = VariancePreservingSDE(beta_min=0.1, beta_max=20.0, T=T1)

    t = torch.ones(batch_size, device=device)

    if z is None:
        init_x = torch.randn(batch_size, 1, 28, 28, device=device) \
        * sde.marginal_prob_std(t)[:, None, None, None]
    else:
        init_x = z
        
    shape = init_x.shape

    def score_eval_wrapper(sample, time_steps):
        sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
        time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))   

        with torch.no_grad():    
            score = score_model(sample, time_steps)
            return score.cpu().numpy().reshape((-1,)).astype(np.float64)
    
    def ode_func(t, x):        
        time_steps = np.ones((shape[0],)) * t   

        g = sde.g(torch.tensor(t),np.ones((shape[0],))).cpu().numpy()

        return  -0.5 * (g**2) * score_eval_wrapper(x, time_steps)


    res = integrate.solve_ivp(ode_func, (1., eps), init_x.reshape(-1).cpu().numpy(), rtol=rtol, atol=atol, method='RK45')  
    print(f"Number of function evaluations: {res.nfev}")
    x = torch.tensor(res.y[:, -1], device=device).reshape(shape)

    return x


def prior_likelihood(z, sigma):
  shape = z.shape
  N = np.prod(shape[1:])
  return -N / 2. * torch.log(2*np.pi*sigma**2) - torch.sum(z**2, dim=(1,2,3)) / (2 * sigma**2)

def ode_likelihood(x, 
                   score_model,
                   batch_size=64, 
                   eps=1e-5):
    
    epsilon = torch.randn_like(x)
    device='cuda'
    T1 = torch.nn.Parameter(torch.FloatTensor([1.0]), requires_grad=False)
        
    sde = VariancePreservingSDE(beta_min=0.1, beta_max=20.0, T=T1)
  
    def divergence_eval(sample, time_steps, epsilon):      
      
        with torch.enable_grad():
            sample.requires_grad_(True)
            score_e = torch.sum(score_model(sample, time_steps) * epsilon)
            #print("bkjs3")
            grad_score_e = torch.autograd.grad(score_e, sample)[0]
            #print("bkjs3")
        return torch.sum(grad_score_e * epsilon, dim=(1, 2, 3))    
    
    shape = x.shape

    def score_eval_wrapper(sample, time_steps):
        sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
        time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))    
        with torch.no_grad():    
            score = score_model(sample, time_steps)
        return score.cpu().numpy().reshape((-1,)).astype(np.float64)
    
    def divergence_eval_wrapper(sample, time_steps):
        with torch.no_grad():
            sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
            #print("bkjs1")
            time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], )) 
            #print("bkjs2")
            div = divergence_eval(sample, time_steps, epsilon)
            #print("bkjs4")
            return div.cpu().numpy().reshape((-1,)).astype(np.float64)
        
    def ode_func(t, x):
        time_steps = np.ones((shape[0],)) * t    
        sample = x[:-shape[0]]
        #print(sample.shape)
        logp = x[-shape[0]:]
        g = sde.g(torch.tensor(t), torch.tensor(sample)).cpu().numpy()
        sample_grad = -0.5 * g**2 * score_eval_wrapper(sample, time_steps)
        #print(sample.shape, time_steps.shape)
        #print(divergence_eval_wrapper(sample, time_steps).shape , g.shape)
        logp_grad = -0.5 * g**2 * divergence_eval_wrapper(sample, time_steps)
        #print("bkjs5")
        return np.concatenate([sample_grad, logp_grad], axis=0)


    init = np.concatenate([x.cpu().numpy().reshape((-1,)), np.zeros((shape[0],))], axis=0)
    res = integrate.solve_ivp(ode_func, (eps, 1.), init, rtol=1e-5, atol=1e-5, method='RK45')  
    zp = torch.tensor(res.y[:, -1], device=device)
    z = zp[:-shape[0]].reshape(shape)
    delta_logp = zp[-shape[0]:].reshape(shape[0])
    sigma_max = sde.marginal_prob_std( torch.tensor(1., device=device))
    prior_logp = prior_likelihood(z, sigma_max)
    bpd = -(prior_logp + delta_logp) / np.log(2)
    N = np.prod(shape[1:])
    bpd = bpd / N + 8.
    return z, bpd

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import _calculate_fan_in_and_fan_out


class Swish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.sigmoid(x) * x


def group_norm(out_ch):
    return nn.GroupNorm(num_groups=32, num_channels=out_ch, eps=1e-6, affine=True)


def upsample(in_ch, with_conv):
    up = nn.Sequential()
    up.add_module('up_nn', nn.Upsample(scale_factor=2, mode='nearest'))
    if with_conv:
        up.add_module('up_conv', conv2d(in_ch, in_ch, kernel_size=(3, 3), stride=1))
    return up


def downsample(in_ch, with_conv):
    if with_conv:
        down = conv2d(in_ch, in_ch, kernel_size=(3, 3), stride=2)
    else:
        down = nn.AvgPool2d(2, 2)
    return down


class ResidualBlock(nn.Module):
    def __init__(self, in_ch, temb_ch, out_ch=None, conv_shortcut=False, dropout=0., normalize=group_norm, act=Swish()):
        super().__init__()
        self.in_ch = in_ch
        self.temb_ch = temb_ch
        self.out_ch = out_ch if out_ch is not None else in_ch
        self.conv_shortcut = conv_shortcut
        self.dropout = dropout
        self.act = act

        self.temb_proj = dense(temb_ch, out_ch)
        self.norm1 = normalize(in_ch) if normalize is not None else nn.Identity()
        self.conv1 = conv2d(in_ch, out_ch)
        self.norm2 = normalize(out_ch) if normalize is not None else nn.Identity()
        self.dropout = nn.Dropout2d(p=dropout) if dropout > 0. else nn.Identity()
        self.conv2 = conv2d(out_ch, out_ch, init_scale=0.)
        if in_ch != out_ch:
            if conv_shortcut:
                self.shortcut = conv2d(in_ch, out_ch)
            else:
                self.shortcut = conv2d(in_ch, out_ch, kernel_size=(1, 1), padding=0)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x, temb):
        # forward conv1
        h = x
        h = self.act(self.norm1(h))
        h = self.conv1(h)

        # add in timestep embedding
        h = h + self.temb_proj(self.act(temb))[:, :, None, None]

        # forward conv2
        h = self.act(self.norm2(h))
        h = self.dropout(h)
        h = self.conv2(h)

        # shortcut
        x = self.shortcut(x)

        # combine and return
        assert x.shape == h.shape
        return x + h


class SelfAttention(nn.Module):

    def __init__(self, in_channels, normalize=group_norm):
        super().__init__()
        self.in_channels = in_channels
        self.attn_q = conv2d(in_channels, in_channels, kernel_size=1, stride=1, padding=0)
        self.attn_k = conv2d(in_channels, in_channels, kernel_size=1, stride=1, padding=0)
        self.attn_v = conv2d(in_channels, in_channels, kernel_size=1, stride=1, padding=0)
        self.proj_out = conv2d(in_channels, in_channels, kernel_size=1, stride=1, padding=0, init_scale=0.)
        self.softmax = nn.Softmax(dim=-1)
        if normalize is not None:
            self.norm = normalize(in_channels)
        else:
            self.norm = nn.Identity()

    # noinspection PyUnusedLocal
    def forward(self, x, temp=None):
        """ t is not used """
        _, C, H, W = x.size()

        h = self.norm(x)
        q = self.attn_q(h).view(-1, C, H * W)
        k = self.attn_k(h).view(-1, C, H * W)
        v = self.attn_v(h).view(-1, C, H * W)

        attn = torch.bmm(q.permute(0, 2, 1), k) * (int(C) ** (-0.5))
        attn = self.softmax(attn)

        h = torch.bmm(v, attn.permute(0, 2, 1))
        h = h.view(-1, C, H, W)
        h = self.proj_out(h)

        assert h.shape == x.shape
        return x + h


def _calculate_correct_fan(tensor, mode):
    
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out', 'fan_avg']
    if mode not in valid_modes:
        raise ValueError("Mode {} not supported, please use one of {}".format(mode, valid_modes))

    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    return fan_in if mode == 'fan_in' else fan_out


def kaiming_uniform_(tensor, gain=1., mode='fan_in'):
   
    fan = _calculate_correct_fan(tensor, mode)
    # gain = calculate_gain(nonlinearity, a)
    var = gain / max(1., fan)
    bound = math.sqrt(3.0 * var)  # Calculate uniform bounds from standard deviation
    with torch.no_grad():
        return tensor.uniform_(-bound, bound)


def variance_scaling_init_(tensor, scale):
    return kaiming_uniform_(tensor, gain=1e-10 if scale == 0 else scale, mode='fan_avg')


def dense(in_channels, out_channels, init_scale=1.):
    lin = nn.Linear(in_channels, out_channels)
    variance_scaling_init_(lin.weight, scale=init_scale)
    nn.init.zeros_(lin.bias)
    return lin


def conv2d(in_planes, out_planes, kernel_size=(3, 3), stride=1, dilation=1, padding=1, bias=True, padding_mode='zeros',
           init_scale=1.):
    conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation,
                     bias=bias, padding_mode=padding_mode)
    variance_scaling_init_(conv.weight, scale=init_scale)
    if bias:
        nn.init.zeros_(conv.bias)
    return conv


def get_sinusoidal_positional_embedding(timesteps: torch.LongTensor, embedding_dim: int):

    assert len(timesteps.size()) == 1
    timesteps = timesteps.to(torch.get_default_dtype())
    device = timesteps.device

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float, device=device) * -emb)
    emb = timesteps[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)  # bsz x embd
    if embedding_dim % 2 == 1:  # zero pad
        emb = F.pad(emb, (0, 1), "constant", 0)
    assert list(emb.size()) == [timesteps.size(0), embedding_dim]
    return emb


class TimestepEmbedding(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, act=Swish()):
        super().__init__()

        self.embedding_dim = embedding_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        self.main = nn.Sequential(
            dense(embedding_dim, hidden_dim),
            act,
            dense(hidden_dim, output_dim),
        )

    def forward(self, temp):
        temb = get_sinusoidal_positional_embedding(temp, self.embedding_dim)
        temb = self.main(temb)
        return temb


class UNet(nn.Module):
    def __init__(self,
                 input_channels,
                 input_height,
                 ch,
                 output_channels=None,
                 ch_mult=(1, 2, 4, 8),
                 num_res_blocks=2,
                 attn_resolutions=(16,),
                 dropout=0.,
                 resamp_with_conv=True,
                 act=Swish(),
                 normalize=group_norm,
                 ):
        super().__init__()
        self.input_channels = input_channels
        self.input_height = input_height
        self.ch = ch
        self.output_channels = output_channels = input_channels if output_channels is None else output_channels
        self.ch_mult = ch_mult
        self.num_res_blocks = num_res_blocks
        self.attn_resolutions = attn_resolutions
        self.dropout = dropout
        self.resamp_with_conv = resamp_with_conv
        self.act = act
        self.normalize = normalize

        # init
        self.num_resolutions = num_resolutions = len(ch_mult)
        in_ht = input_height
        in_ch = input_channels
        temb_ch = ch * 4

        assert in_ht % 2 ** (num_resolutions - 1) == 0, "input_height doesn't satisfy the condition"

        # Timestep embedding
        self.temb_net = TimestepEmbedding(
            embedding_dim=ch,
            hidden_dim=temb_ch,
            output_dim=temb_ch,
            act=act,
        )

        # Downsampling
        self.begin_conv = conv2d(in_ch, ch)
        unet_chs = [ch]
        in_ht = in_ht
        in_ch = ch
        down_modules = []
        for i_level in range(num_resolutions):
            # Residual blocks for this resolution
            
            block_modules = {}
            out_ch = ch * ch_mult[i_level]
            for i_block in range(num_res_blocks):

                block_modules['{}a_{}a_block'.format(i_level, i_block)] = \
                    ResidualBlock(
                        in_ch=in_ch,
                        temb_ch=temb_ch,
                        out_ch=out_ch,
                        dropout=dropout,
                        act=act,
                        normalize=normalize,
                    )
                if in_ht in attn_resolutions:
                    block_modules['{}a_{}b_attn'.format(i_level, i_block)] = SelfAttention(out_ch, normalize=normalize)
                unet_chs += [out_ch]
                in_ch = out_ch
            # Downsample
            if i_level != num_resolutions - 1:
                block_modules['{}b_downsample'.format(i_level)] = downsample(out_ch, with_conv=resamp_with_conv)
                in_ht //= 2
                unet_chs += [out_ch]
            # convert list of modules to a module list, and append to a list
            down_modules += [nn.ModuleDict(block_modules)]
        # conver to a module list
        self.down_modules = nn.ModuleList(down_modules)

        # Middle
        mid_modules = []
        mid_modules += [
            ResidualBlock(in_ch, temb_ch=temb_ch, out_ch=in_ch, dropout=dropout, act=act, normalize=normalize)]
        mid_modules += [SelfAttention(in_ch, normalize=normalize)]
        mid_modules += [
            ResidualBlock(in_ch, temb_ch=temb_ch, out_ch=in_ch, dropout=dropout, act=act, normalize=normalize)]
        self.mid_modules = nn.ModuleList(mid_modules)

        # Upsampling
        up_modules = []
        for i_level in reversed(range(num_resolutions)):
            # Residual blocks for this resolution
            block_modules = {}
            out_ch = ch * ch_mult[i_level]
            for i_block in range(num_res_blocks + 1):
                block_modules['{}a_{}a_block'.format(i_level, i_block)] = \
                    ResidualBlock(
                        in_ch=in_ch + unet_chs.pop(),
                        temb_ch=temb_ch,
                        out_ch=out_ch,
                        dropout=dropout,
                        act=act,
                        normalize=normalize)
                if in_ht in attn_resolutions:
                    block_modules['{}a_{}b_attn'.format(i_level, i_block)] = SelfAttention(out_ch, normalize=normalize)
                in_ch = out_ch
            # Upsample
            if i_level != 0:
                block_modules['{}b_upsample'.format(i_level)] = upsample(out_ch, with_conv=resamp_with_conv)
                in_ht *= 2
            # convert list of modules to a module list, and append to a list
            up_modules += [nn.ModuleDict(block_modules)]
        # conver to a module list
        self.up_modules = nn.ModuleList(up_modules)
        assert not unet_chs

        # End
        self.end_conv = nn.Sequential(
            normalize(in_ch),
            self.act,
            conv2d(in_ch, output_channels, init_scale=0.),
        )

    # noinspection PyMethodMayBeStatic
    def _compute_cond_module(self, module, x, temp):
        for m in module:
            x = m(x, temp)
        return x

    # noinspection PyArgumentList,PyShadowingNames
    def forward(self, x, temp):
        # Init
        B, C, H, W = x.size()

        # Timestep embedding
        temb = self.temb_net(temp)
        assert list(temb.shape) == [B, self.ch * 4]

        # Downsampling
        hs = [self.begin_conv(x)]
        for i_level in range(self.num_resolutions):
            # Residual blocks for this resolution
            block_modules = self.down_modules[i_level]
            for i_block in range(self.num_res_blocks):
                resnet_block = block_modules['{}a_{}a_block'.format(i_level, i_block)]
                h = resnet_block(hs[-1], temb)
                if h.size(2) in self.attn_resolutions:
                    attn_block = block_modules['{}a_{}b_attn'.format(i_level, i_block)]
                    h = attn_block(h, temb)
                hs.append(h)
            # Downsample
            if i_level != self.num_resolutions - 1:
                downsample = block_modules['{}b_downsample'.format(i_level)]
                hs.append(downsample(hs[-1]))

        # Middle
        h = hs[-1]
        h = self._compute_cond_module(self.mid_modules, h, temb)

        # Upsampling
        for i_idx, i_level in enumerate(reversed(range(self.num_resolutions))):
            # Residual blocks for this resolution
            block_modules = self.up_modules[i_idx]
            for i_block in range(self.num_res_blocks + 1):
                resnet_block = block_modules['{}a_{}a_block'.format(i_level, i_block)]
                h = resnet_block(torch.cat([h, hs.pop()], axis=1), temb)
                if h.size(2) in self.attn_resolutions:
                    attn_block = block_modules['{}a_{}b_attn'.format(i_level, i_block)]
                    h = attn_block(h, temb)
            # Upsample
            if i_level != 0:
                upsample = block_modules['{}b_upsample'.format(i_level)]
                h = upsample(h)
        assert not hs

        # End
        h = self.end_conv(h)
        assert list(h.size()) == [x.size(0), self.output_channels, x.size(2), x.size(3)]
        return h


In [ ]:
from pytorch_lightning.loggers import WandbLogger

In [ ]:
wandb_logger = WandbLogger(project='MNIST', log_model='all')

wandb: Currently logged in as: sdm8499 (sdm8499pro). Use `wandb login --relogin` to force relogin


In [ ]:
import os
import json
import argparse
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np

#from model import UNet
#from diffusion import VariancePreservingSDE, ode_sampler , ode_likelihood
#import dataset as dataset

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

class ScoreBasedDiffusion(pl.LightningModule):

    def __init__(self, conf):
        super().__init__()

        self.conf  = conf
        self.save_hyperparameters()

        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.training_step_outputs =[]

        self.model = UNet(input_channels= self.conf.model.in_channel,
                          input_height=self.conf.model.height,
                          ch= self.conf.model.ch,
                          ch_mult=self.conf.model.channel_multiplier,
                          num_res_blocks=self.conf.model.n_res_blocks,
                          attn_resolutions= (16,),
                          resamp_with_conv=True,
                          dropout=self.conf.model.dropout,
                          )

        T1 = torch.nn.Parameter(torch.FloatTensor([self.conf.model.sde.T]), requires_grad=False)
        
        self.sde = VariancePreservingSDE(beta_min=0.1, beta_max=20.0, T=T1)

    def forward(self , x, t):
        score = self.model(x , t)
        return score

    def setup(self, stage):
        self.train_set, self.valid_set , self.test_set = get_train_data(self.conf)


    def configure_optimizers(self):
        if self.conf.training.optimizer.type == 'adam':
            optimizer = optim.Adam(self.model.parameters(), lr=self.conf.training.optimizer.lr)
        else:
            raise NotImplementedError

        return optimizer


    def training_step(self, batch, batch_nb):

        eps = 1e-5

        x , _ = batch

        random_t = torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps  
        z = torch.randn_like(x)
        std = self.sde.marginal_prob_std(random_t)
        perturbed_x = x + z * std[:, None, None, None]
        score = self.model(perturbed_x, random_t)
        loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1,2,3)))
        tensorboard_logs = {'train_loss': loss}
        self.log('train_loss', loss)

        
        #x = (x * 255. + torch.rand_like(x)) / 256.    
        #_, bpd = ode_likelihood(x, self.model, x.shape[0], eps)

        self.training_step_outputs.append({'train_loss': loss})# , 'bpd': bpd.sum() , 'bpd_items': bpd.shape[0]})

        return {'loss': loss, 'log': tensorboard_logs}


    def on_train_epoch_end(self):
      avg_loss         = torch.stack([x['train_loss'] for x in self.training_step_outputs]).mean()   
      #all_bpd =torch.stack([x['bpd'] for x in self.training_step_outputs]).sum()

      #all_items = torch.stack([torch.tensor(x['bpd_items']) for x in self.training_step_outputs]).sum()
      self.log('avg_train_loss', avg_loss)
      #wandb.log('avg_bpd', all_bpd/all_items)
      
      return {'avg_train_loss': avg_loss}
    

    def train_dataloader(self):

        train_loader = DataLoader(self.train_set,
                                batch_size=self.conf.training.dataloader.batch_size,
                                shuffle=True,
                                num_workers=self.conf.training.dataloader.num_workers,
                                pin_memory=True,
                                drop_last=self.conf.training.dataloader.drop_last)
        
        print(len(train_loader))

        return train_loader


    def val_dataloader(self):
        valid_loader = DataLoader(self.valid_set,
                                  batch_size=self.conf.validation.dataloader.batch_size,
                                  shuffle=False,
                                  num_workers=self.conf.validation.dataloader.num_workers,
                                  pin_memory=True,
                                  drop_last=self.conf.validation.dataloader.drop_last)

        print(len(valid_loader))
        return valid_loader
    
    def test_dataloader(self):

        test_loader = DataLoader(self.test_set,
                                batch_size=self.conf.test.dataloader.batch_size,
                                shuffle=False,
                                num_workers=self.conf.test.dataloader.num_workers,
                                pin_memory=True,
                                drop_last=self.conf.test.dataloader.drop_last)
        
        return test_loader

    
    def validation_step(self, batch, batch_nb):

        eps = 1e-5
        x , _ = batch

        random_t = torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps  
        z = torch.randn_like(x)
        std = self.sde.marginal_prob_std(random_t)
        perturbed_x = x + z * std[:, None, None, None]
        score = self.model(perturbed_x, random_t)
        loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1,2,3)))

        x = (x * 255. + torch.rand_like(x)) / 256.    
        _, bpd = ode_likelihood(x, self.model, x.shape[0], eps)
    
        #all_bpds += bpd.sum()
        #all_items += bpd.shape[0]

        #avg_bpd = bpd.sum() / bpd.shape[0]

        self.log('val_loss', loss)

        self.validation_step_outputs.append({'val_loss': loss , 'bpd': bpd.sum() , 'bpd_items': bpd.shape[0]})
    
        return {'val_loss': loss}
    

    def on_validation_epoch_end(self):
        avg_loss         = torch.stack([x['val_loss'] for x in self.validation_step_outputs]).mean()
        
        all_bpd =torch.stack([x['bpd'] for x in self.validation_step_outputs]).sum()

        all_items = torch.stack([torch.tensor(x['bpd_items']) for x in self.validation_step_outputs]).sum()

        #tensorboard_logs = {'val_loss': avg_loss , 'avg_bpd': all_bpd/all_items}

        sample_batch_size = 64

        samples = ode_sampler(self.model, sample_batch_size)

        samples = samples.clamp(0.0, 1.0)

        sample_grid = make_grid(samples, nrow=int(np.sqrt(sample_batch_size)))
        wandb.log({'val_images': [wandb.Image(sample_grid, caption='MNIST')]})
        wandb.log({'avg_bpd': all_bpd/all_items})
        self.log('avg_bpd', all_bpd/all_items)

        print(all_bpd, all_items)
        print('avg_bpd', all_bpd/all_items)

        self.validation_step_outputs = []
        
        return {'val_loss': avg_loss , 'avg_bpd' :all_bpd/all_items}
    

    def test_step(self, batch, batch_nb):

        sample_batch_size = 64
        samples = ode_sampler(self.model,
                              sample_batch_size)

        samples = samples.clamp(0.0, 1.0)

        sample_grid = make_grid(samples, nrow=int(np.sqrt(sample_batch_size)))
        #self.logger.experiment.add_image(f'generated_images', sample_grid, self.current_epoch)


        x = (x * 255. + torch.rand_like(x)) / 256.    
        _, bpd = ode_likelihood(x, self.model,
                                x.shape[0], eps=1e-5)

        self.log('testbpd', bpd)
    
        self.test_step_outputs.append({'bpd': bpd.sum() , 'bpd_items': bpd.shape[0]})

        return {'bpd': bpd.sum() , 'bpd_items': bpd.shape[0]}


    def on_test_epoch_end(self):

      all_bpd =torch.stack([x['bpd'] for x in self.test_step_outputs]).sum()
      all_items = torch.stack([torch.tensor(x['bpd_items']) for x in self.test_step_outputs]).sum()
      tensorboard_logs = {'test_avg_bpd': all_bpd/all_items}
      self.log('test_avg_bpd', all_bpd/all_items)

      return {'log': tensorboard_logs}
 

if __name__ == "__main__":


    path_to_config = "/content/mnist.json"

    with open(path_to_config, 'r') as f:
        conf = json.load(f)

    conf = obj(conf)

    score_based_diffusion_model = ScoreBasedDiffusion(conf)

    checkpoint_callback = ModelCheckpoint(dirpath=os.path.join("/content/sample_data/ckptdir"),
                                           filename="best",
                                            monitor='avg_bpd',
                                            mode = 'min', 
                                            verbose=False,
                                            save_last=True,
                                            save_top_k=1,
                                            save_weights_only=True,
                                            
                                            )
    
    trainer = pl.Trainer(fast_dev_run= False,logger=wandb_logger, accelerator ="gpu",max_epochs= 25,num_nodes= 1,check_val_every_n_epoch=20 ,callbacks=checkpoint_callback)

    trainer.fit(score_based_diffusion_model)

    

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                  | Params
------------------------------------------------
0 | model | UNet                  | 1.5 M 
1 | sde   | VariancePreservingSDE | 1     
------------------------------------------------
1.5 M     Trainable params
1         Non-trainable params
1.5 M     Total params
6.157     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

94
Number of function evaluations: 14
tensor(601.6431, device='cuda:0', dtype=torch.float64) tensor(64)
avg_bpd tensor(9.4007, device='cuda:0', dtype=torch.float64)
1781


Training: 0it [00:00, ?it/s]

In [ ]:
wandb.finish()

NameError: ignored